<a href="https://colab.research.google.com/github/phamduyaaaa/Robot-Kinematic/blob/main/Ackerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import os
import cv2

class AckermannCar:
    def __init__(self, start_pos, L):
        self.x, self.y = start_pos
        self.theta = 0  # hướng đầu xe
        self.v = 0.0    # tốc độ tiến (m/s)
        self.delta = 0.0  # góc lái (rad)
        self.L = L      # chiều dài trục bánh (wheelbase)
        self.trail = []

    def update(self, dt):
        self.x += self.v * math.cos(self.theta) * dt
        self.y += self.v * math.sin(self.theta) * dt
        self.theta += (self.v / self.L) * math.tan(self.delta) * dt
        self.theta %= 2 * math.pi

        self.trail.append((self.x, self.y))
        if len(self.trail) > 1000:
            self.trail.pop(0)


In [1]:
def draw_car_frame(car, frame_num):
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 3)
    ax.set_aspect('equal')
    ax.set_title(f"Ackermann Frame {frame_num}")

    if len(car.trail) > 1:
        trail = np.array(car.trail)
        ax.plot(trail[:, 0], trail[:, 1], 'r-')

    # Vẽ thân xe
    car_length = 0.4
    car_width = 0.2
    rect = patches.Rectangle(
        (car.x - car_length/2, car.y - car_width/2),
        car_length, car_width,
        angle=np.degrees(car.theta),
        linewidth=2, edgecolor='blue', facecolor='lightblue', zorder=5)
    t = plt.matplotlib.transforms.Affine2D().rotate_around(car.x, car.y, car.theta) + ax.transData
    rect.set_transform(t)
    ax.add_patch(rect)

    # Vẽ trục hướng
    ax.arrow(car.x, car.y,
             0.3 * math.cos(car.theta),
             0.3 * math.sin(car.theta),
             head_width=0.05, color='green')

    filename = f"ackermann_frame_{frame_num:04d}.png"
    plt.savefig(filename)
    plt.close()


In [4]:
!mkdir -p ackermann_frames
os.chdir("ackermann_frames")

car = AckermannCar(start_pos=(1.0, 1.0), L=0.5)
dt = 0.05
frames = 150

for i in range(frames):
    # Điều chỉnh điều khiển (giả lập)
    car.v = 1.0  # m/s
    car.delta = 0.2 * math.sin(i * 0.1)  # đánh lái qua lại
    car.update(dt)
    draw_car_frame(car, i)

os.chdir("..")


In [5]:
img_array = []
for i in range(frames):
    filename = f"ackermann_frames/ackermann_frame_{i:04d}.png"
    img = cv2.imread(filename)
    height, width, _ = img.shape
    img_array.append(img)

out = cv2.VideoWriter('ackermann_sim.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      10, (width, height))

for img in img_array:
    out.write(img)
out.release()


In [6]:
from IPython.display import Video
Video("ackermann_sim.mp4", embed=True)
